In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
import os

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

The idea was to use an autoencoder and its reconstruction loss as a filter for testing if an image was put into te pipeline that did not belong to any of the trained classes. I did not succeed...

I have to use numpy since tf.dataset keeps giving me label errors. I thought i was doing something wrong but later i found a guide that did it that way and for that person it worked. I never resolved it and had to switch over to using numpy instead.

In [26]:
from PIL import Image
import os
import numpy as np

# Define the directory containing your image data
# data_dir = 'EARLY_BLIGHT'
data_dir = r"C:\Users\Magnus\Desktop\code\timeSeries\EARLY_BLIGHT\early_blight"

# List all image files in the directory
image_files = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]

# Initialize an empty list to store the image data
image_data = []

# Loop through the image files and load them into NumPy arrays
for image_file in image_files:
    img = Image.open(image_file)
    img = img.resize((256, 256))  # Resize images to your desired dimensions
    img_array = np.array(img)
    image_data.append(img_array)

# Convert the list of NumPy arrays to a single NumPy array
image_data = np.array(image_data)

# Normalize the pixel values to the range [0, 1] if needed
images = image_data / 255.0

labels = np.ones(len(image_files))

# Check the shape of the loaded data
print("Shape of images:", images.shape)
print('Shape of labels', labels.shape)

Shape of images: (996, 256, 256, 3)
Shape of labels (996,)


In [53]:
import os

data_dir = r"C:\Users\Magnus\Desktop\code\timeSeries\EARLY_BLIGHT\early_blight_test"

# List all image files in the directory
image_files = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]

# Initialize an empty list to store the image data and corresponding file names
image_data = []
file_names = []

# Loop through the image files, load them into NumPy arrays, and store file names
for image_file in image_files:
    img = Image.open(image_file)
    img = img.resize((256, 256))  # Resize images to your desired dimensions
    img_array = np.array(img)
    image_data.append(img_array)
    file_name = os.path.basename(image_file)  # Extract the file name
    file_names.append(file_name)

# Convert the list of NumPy arrays to a single NumPy array
image_data = np.array(image_data)

# Normalize the pixel values to the range [0, 1] if needed
images_test = image_data / 255.0

# Use file names as labels
labels_test = file_names

# Check the shape of the loaded data
print("Shape of images:", images_test.shape)
print('Shape of labels:', len(labels_test))
print('Image File names:', file_names)


Shape of images: (9, 256, 256, 3)
Shape of labels: 9
Image File names: ['EB_1.JPG', 'EB_2.JPG', 'EB_3.JPG', 'LB_1.JPG', 'LB_2.JPG', 'LB_3.JPG', 'y_1.JPG', 'y_2.JPG', 'y_3.JPG']


In [54]:
file_names
test_labels = [1,1,1,2,2,2,3,3,3]
test_labels_arr = np.array(test_labels)

In [28]:
print(len(images))
print(len(labels))
print(len(images_test))
print(len(labels_test))

996
996
6
6


A common autoencoder architecture

In [27]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(256, 256, 3)),
      layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(64, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(128, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()


autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(images, images,
                epochs=10,
                shuffle=True)

Epoch 1/10
32/32 [==============================] - 117s 4s/step - loss: 0.0447
Epoch 2/10
32/32 [==============================] - 121s 4s/step - loss: 0.0203
Epoch 3/10
32/32 [==============================] - 121s 4s/step - loss: 0.0103
Epoch 4/10
32/32 [==============================] - 121s 4s/step - loss: 0.0084
Epoch 5/10
32/32 [==============================] - 119s 4s/step - loss: 0.0077
Epoch 6/10
32/32 [==============================] - 120s 4s/step - loss: 0.0074
Epoch 7/10
32/32 [==============================] - 118s 4s/step - loss: 0.0071
Epoch 8/10
32/32 [==============================] - 119s 4s/step - loss: 0.0070
Epoch 9/10
32/32 [==============================] - 119s 4s/step - loss: 0.0068
Epoch 10/10
32/32 [==============================] - 118s 4s/step - loss: 0.0068


In [56]:
reconstructions = autoencoder.predict(images_test)
test_loss = tf.keras.losses.mae(reconstructions, images_test)
test_loss

1/1 [==============================] - 0s 279ms/step


<tf.Tensor: shape=(9, 256, 256), dtype=float64, numpy=
array([[[0.23638134, 0.15132243, 0.28400104, ..., 0.01050796,
         0.07478106, 0.08139582],
        [0.12345941, 0.13153644, 0.1701395 , ..., 0.09368209,
         0.02750916, 0.1352342 ],
        [0.18685597, 0.28792092, 0.18222582, ..., 0.0500705 ,
         0.01887463, 0.03643409],
        ...,
        [0.02683432, 0.03428159, 0.03310477, ..., 0.0480013 ,
         0.03099233, 0.06442416],
        [0.03318992, 0.07462111, 0.04538211, ..., 0.04855513,
         0.07777776, 0.02412888],
        [0.0332695 , 0.03242588, 0.05320393, ..., 0.13460113,
         0.02691292, 0.11565174]],

       [[0.14675852, 0.15878056, 0.16537416, ..., 0.04812819,
         0.09569802, 0.11155248],
        [0.13094712, 0.1534641 , 0.16770033, ..., 0.02392326,
         0.01148375, 0.3372366 ],
        [0.13906558, 0.16810625, 0.17914663, ..., 0.00437351,
         0.07138054, 0.02262161],
        ...,
        [0.0250698 , 0.02287699, 0.03236139, ..., 0.1

In [61]:
# Initialize an empty NumPy array to store the per-image MAE values
mae_per_image = np.zeros(images_test.shape[0])

# Loop through each image and calculate the MAE
for i in range(images_test.shape[0]):
    mae_per_image[i] = np.mean(np.abs(reconstructions[i] - images_test[i]))

# Now, 'mae_per_image' contains one MAE value per image
mae_per_image

array([0.08198355, 0.06307536, 0.07137079, 0.05261784, 0.06301957,
       0.06448122, 0.05561505, 0.07138394, 0.05130228])

In [62]:
reconstructions = autoencoder.predict(images_test)
test_loss_per_image = tf.reduce_mean(tf.reduce_mean(tf.reduce_mean(tf.abs(reconstructions - images_test), axis=1), axis=1), axis=1)
test_loss_per_image

<tf.Tensor: shape=(9,), dtype=float64, numpy=
array([0.08198355, 0.06307536, 0.07137079, 0.05261784, 0.06301957,
       0.06448122, 0.05561505, 0.07138394, 0.05130228])>

I am almost sure that the images are in the order that they were in the original folder. The first 3 images are of the same class that the model trained on reducing and then reconstructing whilst the other 6 images or 2 before unseen classes. As you can see, again the reconstruction error is highest among the images that the model has already seen so i am doing something wrong.

Its pointless to calculate a threshold since i already know the pattern is in the wrong direction.

I am trying a different type of autoencotder to see if that improves things

In [2]:
# data_dir = 'EARLY_BLIGHT'
data_dir = r"C:\Users\Magnus\Desktop\code\timeSeries\EARLY_BLIGHT\early_blight"

# List all image files in the directory
image_files = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]

# Initialize an empty list to store the image data
image_data = []

# Loop through the image files and load them into NumPy arrays
for image_file in image_files:
    img = Image.open(image_file)
    img = img.resize((256, 256))  # Resize images to your desired dimensions
    img_array = np.array(img)
    image_data.append(img_array)

# Convert the list of NumPy arrays to a single NumPy array
image_data = np.array(image_data)

# Normalize the pixel values to the range [0, 1] if needed
images = image_data / 255.0

labels = np.ones(len(image_files))

# Check the shape of the loaded data
print("Shape of images:", images.shape)
print('Shape of labels', labels.shape)

Shape of images: (995, 256, 256, 3)
Shape of labels (995,)


In [3]:
data_dir = r"C:\Users\Magnus\Desktop\code\timeSeries\early_blight_test\EB"

# List all image files in the directory
image_files = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]

# Initialize an empty list to store the image data and corresponding file names
image_data = []
file_names = []

# Loop through the image files, load them into NumPy arrays, and store file names
for image_file in image_files:
    img = Image.open(image_file)
    img = img.resize((256, 256))  # Resize images to your desired dimensions
    img_array = np.array(img)
    image_data.append(img_array)
    file_name = os.path.basename(image_file)  # Extract the file name
    file_names.append(file_name)

# Convert the list of NumPy arrays to a single NumPy array
image_data = np.array(image_data)

# Normalize the pixel values to the range [0, 1] if needed
images_test = image_data / 255.0

# Use file names as labels
labels_test = file_names

# Check the shape of the loaded data
print("Shape of images:", images_test.shape)
print('Shape of labels:', len(labels_test))
print('Image File names:', file_names)


file_names
test_labels = [1,1,1,2,2,2,3,3,3]
test_labels_arr = np.array(test_labels)

Shape of images: (9, 256, 256, 3)
Shape of labels: 9
Image File names: ['EB_1.JPG', 'EB_2.JPG', 'EB_3.JPG', 'LB_1.JPG', 'LB_2.JPG', 'LB_3.JPG', 'y_1.JPG', 'y_2.JPG', 'y_3.JPG']


In [4]:
print(len(images))
print(images[0].shape)
print(len(labels))
print(len(images_test))
print(images_test[0].shape)
print(len(labels_test))

995
(256, 256, 3)
995
9
(256, 256, 3)
9


I try a different architecture to see if it improves the reconstruction

In [26]:
# Define input shape
input_shape = (256, 256, 3)

# Build encoder
encoder = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=2, activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='relu'),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu')
])

latent_shape = (256,)


# Build decoder
decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(units=256, activation='relu', input_shape=latent_shape),
    tf.keras.layers.Dense(units=4096, activation='relu'),
    tf.keras.layers.Reshape(target_shape=(16, 16, 16)),
    tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, activation='relu', padding='same'),
    tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=2, activation='sigmoid', padding='same')
])

# Build autoencoder
autoencoder = tf.keras.Sequential([encoder, decoder])

In [29]:
autoencoder.summary()

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_44 (Sequential)  (None, 256)               15768864  
                                                                 
 sequential_45 (Sequential)  (None, 256, 256, 3)       1230147   
                                                                 
Total params: 16999011 (64.85 MB)
Trainable params: 16999011 (64.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile model
autoencoder.compile(optimizer='adam', loss='mse')

# Train model
history = autoencoder.fit(
    images,images,
    epochs=15,
)


Epoch 1/15
32/32 [==============================] - 29s 857ms/step - loss: 0.0360
Epoch 2/15
32/32 [==============================] - 28s 864ms/step - loss: 0.0322
Epoch 3/15
32/32 [==============================] - 26s 803ms/step - loss: 0.0285
Epoch 4/15
32/32 [==============================] - 22s 697ms/step - loss: 0.0254
Epoch 5/15
32/32 [==============================] - 25s 768ms/step - loss: 0.0223
Epoch 6/15
32/32 [==============================] - 27s 840ms/step - loss: 0.0201
Epoch 7/15
32/32 [==============================] - 25s 794ms/step - loss: 0.0183
Epoch 8/15
32/32 [==============================] - 25s 787ms/step - loss: 0.0180
Epoch 9/15
32/32 [==============================] - 25s 772ms/step - loss: 0.0165
Epoch 10/15
32/32 [==============================] - 26s 798ms/step - loss: 0.0157
Epoch 11/15
32/32 [==============================] - 25s 773ms/step - loss: 0.0152
Epoch 12/15
32/32 [==============================] - 25s 789ms/step - loss: 0.0146
Epoch 13/15
3

In [25]:
reconstructions = autoencoder.predict(images_test)
test_loss_per_image = tf.reduce_mean(tf.reduce_mean(tf.reduce_mean(tf.abs(reconstructions - images_test), axis=1), axis=1), axis=1)
test_loss_per_image

1/1 [==============================] - 0s 484ms/step


<tf.Tensor: shape=(9,), dtype=float64, numpy=
array([0.10340108, 0.08843391, 0.10330547, 0.07726714, 0.10288679,
       0.09639677, 0.07369517, 0.09185791, 0.05853234])>

I am almost sure that the images are in the order that they were in the original folder. The first 3 images are of the same class that the model trained on reducing and then reconstructing whilst the other 6 images or 2 before unseen classes. As you can see, again the reconstruction error is highest among the images that the model has already seen so i am doing something wrong.

Its pointless to calculate a threshold since i already know the pattern is in the wrong direction.